In [44]:
import pandas as pd

In [45]:
titanic = pd.read_csv("train.csv")
titanic_test = pd.read_csv("test.csv")
gender_submission = pd.read_csv("gender_submission.csv")

## Prepare the Data for Machine Learning Algorithms

### Data Cleaning

In this step, two approaches were worked upon. I seperated code steps into modules for readibilty. Here, I will import pipeline.ipynb for clean the data up. If you want to see what code produced this output, check pipeline.ipynb file. And for seeing another approach without pipelines, see file data_cleaning_w/o_pipeline.ipynb

In [46]:
# import pipeline.ipynb
%run pipeline.ipynb

In [47]:
titanic_num = ['Age', 'Fare']
titanic_cat = ['Sex', 'Embarked']

In [48]:
titanic_prepared = full_pipeline(titanic_num, titanic_cat)
titanic_clean = titanic_prepared.fit_transform(titanic)

In [49]:
titanic_clean.shape

(891, 7)

In [50]:
# the shape of original dataframe has been changed
titanic_clean.shape != titanic.shape

True

In [51]:
#titanic_df = pd.DataFrame(titanic_clean, columns=['Age', 'Fare', 'Sex', 'Embarked'])